In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import glob 
%matplotlib qt
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Camera Callibration

## Distortion Correction

In [2]:
# Read in calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Setup object points
nx = 9
ny = 6

# Array to store object points and image points from all images
objpoints = [] # 3D points in real world space, undistored image
imgpoints = [] # 2D points in image plane, distorted image

# Prepare object points
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

# Loop through images and add points
for fname in images:
    # Read each image
    img = cv2.imread(fname)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

    # If corners are found, add object points, image points
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

In [3]:
# Do camera callibration with object points and image points
def cal_undistort(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

## Test camera callibration

In [4]:
# Function to compare original and processed images
def image_compare(img_name, src, dst, output_path, cmap):
    src = cv2.cvtColor(src, cv2.COLOR_RGB2BGR)
    if cmap == None:
        dst = cv2.cvtColor(dst, cv2.COLOR_RGB2BGR)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.set_title('Original Image', fontsize=50)
    ax1.imshow(src)
    ax2.set_title('Processed Image', fontsize=50)
    if(cmap == 'gray'):
        ax2.imshow(dst, cmap='gray')
    else:
        ax2.imshow(dst)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.savefig(output_path + img_name)

In [5]:
# Save test image to output
images = glob.glob('test_images/' + '*.jpg')

for fname in images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, objpoints, imgpoints)
    image_compare(fname[11:], img, undistorted, 'output_images/callibration/', None)

# Perspective Transform

In [6]:
# Function to transform an image
def corners_unwarp(undist):    
    # Convert undistorted image 
    gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)

    # Get image size
    img_size = (gray.shape[1], gray.shape[0])

    # Choose offet to calculate for destination image
    offset = 100 #
    
    # Get the 4 points in the source image
    src = np.float32([[585, 455], 
                       [705, 455], 
                       [1130, 720], 
                       [190, 720]])
    
    # Calculate destination points
    dst = np.float32([[offset, 0],
                     [img_size[0] - offset, 0],
                     [img_size[0] - offset, img_size[1]],
                     [offset, img_size[1]]])
    
    # Get the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)

    # Get the inverse perspective transform matrix
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    # Warp the image 
    warped = cv2.warpPerspective(undist, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped, M, Minv

## Test Perspective Transform

In [7]:
# Save test image to output
images = glob.glob('test_images/' + '*.jpg')

for fname in images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, objpoints, imgpoints)
    top_down, perspective_M, perspective_Minv= corners_unwarp(undistorted)
    image_compare(fname[12:], img, top_down, 'output_images/perspective_transform/', None)

# Color and Gradient

## Combine Gradient and Color Threshold Techniques

In [8]:
# Define a function that computes sobel absolute
def abs_sobel_thresh(orient, sobelx, sobely, thresh=(0, 255)):
    # Take the absolute value of the derivative or gradient
    if orient == 'x':
        abs_sobel = np.absolute(sobelx)
    if orient == 'y':
        abs_sobel = np.absolute(sobely)
        
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a mask of 1's where the scaled gradient magnitude is > thresh_min and < thresh_max
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    return grad_binary

# Define a function that computes the magnitude of the gradient
def mag_thresh(sobelx, sobely, mag_thresh=(0, 255)):
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    return mag_binary

# Define a function that computesthe direction of the gradient
def dir_threshold(sobelx, sobely, thresh=(0, np.pi/2)):
    # Take the absolute value of the gradient direction, apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    return dir_binary

# Define funcion to detect lines based on color threshold
def color_threshold(img):
    # Convert to HLS color space and separate the S channel
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2]
    
    # Threshold color channel
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    return s_binary

# Combine all methods
def gradient_color_combined(img, sobel_kernel = 3):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Call defined functions for each type of threshold
    gradx = abs_sobel_thresh('x', sobelx, sobely, thresh=(20, 100))
    grady = abs_sobel_thresh('y', sobelx, sobely, thresh=(20, 100))
    mag_binary = mag_thresh(sobelx, sobely, mag_thresh=(30, 100))
    dir_binary = dir_threshold(sobelx, sobely, thresh=(0.7, 1.3))

    # Combine gradient
    gradient_binary = np.zeros_like(dir_binary)
    gradient_binary[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    # Get color threshold result
    color_binary = color_threshold(img)
    
    # Combine the two binary thresholds
    combined = np.zeros_like(gradient_binary)
    combined[(color_binary == 1) | (gradient_binary == 1)] = 1
    
    return combined

## Test Gradient Color Combined

In [9]:
# Save test image to output
images = glob.glob('test_images/' + '*.jpg')

for fname in images:
    img = cv2.imread(fname)
    undistorted = cal_undistort(img, objpoints, imgpoints)
    combined = gradient_color_combined(undistorted, sobel_kernel=15)
    image_compare(fname[12:], img, combined, 'output_images/color_and_gradient/', 'gray')

# Finding Lanes

In [10]:
# Function to find lanelines
def find_lane_pixels(binary_warped):
    # Take histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Find the peaks of the left and right halves of the histogram
    # These will be starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = midpoint + np.argmax(histogram[midpoint:])
    
    # Number of sliding windows
    nwindows = 9
    
    # Window width
    margin = 100
    
    # Minimum pixels found to recenter window
    minpix = 50
    
    # Window height
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current window position
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Create empty list to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the window one by one
    for window in range(nwindows):
        # Identify window boundaries
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - (window)*window_height
        
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
#         # Draw the windows on the visualization image
#         cv2.rectangle(out_img,(win_xleft_low,win_y_low),
#         (win_xleft_high,win_y_high),(0,255,0), 2) 
#         cv2.rectangle(out_img,(win_xright_low,win_y_low),
#         (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy <= win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox <= win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If pixels > minpix => recenter the next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        
    # Concatenate the arrays of indices
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoid error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]


    return leftx, lefty, rightx, righty

In [11]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

## Test lane detected

In [26]:
# for fname in images:
fname = 'test_images/test2.jpg'
img = cv2.imread(fname)
undist = cal_undistort(img, objpoints, imgpoints)
warped, perspective_M, perspective_Minv = corners_unwarp(undist)
binary_warped = gradient_color_combined(warped, sobel_kernel=15) 

# Create output image to vixualize results
out_img = np.dstack((binary_warped, binary_warped, binary_warped))
leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)    
left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape,leftx, lefty, rightx, righty )

## Visualization ##
# Colors in the left and right lane regions
out_img[lefty, leftx] = [255, 0, 0]
out_img[righty, rightx] = [0, 0, 255]

# Plots the left and right polynomials on the lane lines
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')

plt.imshow(out_img)
plt.savefig('output_images/lane_findings/' + fname[12:])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [13]:
# Search pixels around detected lines
def search_around_poly(binary_warped, pre_left_fit, pre_right_fit):
    # HYPERPARAMETER
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Calculate left_fitx and right_fitx
    left_fitx = pre_left_fit[0]*(nonzeroy**2) + pre_left_fit[1]*nonzeroy + pre_left_fit[2]
    right_fitx = pre_right_fit[0]*(nonzeroy**2) + pre_right_fit[1]*nonzeroy + pre_right_fit[2]
    
    ### TO-DO: Set the area of search based on activated x-values ###
    left_lane_inds = ((nonzerox > (left_fitx - margin)) & (nonzerox < (left_fitx + margin)))
    right_lane_inds = ((nonzerox > (right_fitx - margin)) & (nonzerox < (right_fitx + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty

## Test Search Around

In [25]:
file_name = 'test_images/test2.jpg'
image = cv2.imread(file_name)
margin = 100

undist = cal_undistort(image, objpoints, imgpoints)
warped, perspective_M, perspective_Minv = corners_unwarp(undist)
binary_warped = gradient_color_combined(warped, sobel_kernel=15) 

# Find our lane pixels first
leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)

# Fit a second order polynomial to each using `np.polyfit`
pre_left_fit = np.polyfit(lefty, leftx, 2)
pre_right_fit = np.polyfit(righty, rightx, 2)

# Fit new polynomials
leftx, lefty, rightx, righty = search_around_poly(binary_warped, pre_left_fit, pre_right_fit)
left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

## Visualization ##
# Create an image to draw on and an image to show the selection window
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
window_img = np.zeros_like(out_img)

# Color in left and right line pixels
out_img[lefty, leftx] = [255, 0, 0]
out_img[righty, rightx] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))

right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
lane_area = cv2.addWeighted(out_img, 1, window_img, 0.2, 0)

# Plot the polynomial lines onto the image
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
## End visualization steps ##
    
# View your output
plt.imshow(lane_area)
plt.savefig('output_images/lane_search_around/' + file_name[12:])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [15]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        
        #polynomial coefficients for the most recent fit
        self.current_fit = [] #[np.array([False])]  
        
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        
        #x values for detected line pixels
        self.allx = None  
        
        #y values for detected line pixels
        self.ally = None  

In [16]:
# Draw area between lanes
def draw_lane_area(image):
    # Undistort orginal image
    undist = cal_undistort(image, objpoints, imgpoints)
    
    # Perspetive Transform
    warped, perspective_M, Minv = corners_unwarp(undist)
    
    # Apply gradient and color threshold
    warped = gradient_color_combined(warped, sobel_kernel=15)
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
  
    try:
        left_line.allx, left_line.ally, right_line.allx, right_line.ally = search_around_poly(warped, left_line.best_fit, right_line.best_fit) 
    except:
        left_line.allx, left_line.ally, right_line.allx, right_line.ally = find_lane_pixels(warped)
        
    left_line.best_fit = np.polyfit(left_line.ally, left_line.allx, 2)
    right_line.best_fit = np.polyfit(right_line.ally, right_line.allx, 2)

    left_line.bestx = left_line.best_fit[0]*ploty**2 + left_line.best_fit[1]*ploty + left_line.best_fit[2]
    right_line.bestx = right_line.best_fit[0]*ploty**2 + right_line.best_fit[1]*ploty + right_line.best_fit[2]
    
    # Measure curvature
    left_line.radius_of_curvature, right_line.radius_of_curvature, pos = measure_curvature_real(ploty)
        
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_line.bestx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.bestx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0]), flags=cv2.INTER_LINEAR) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.2, 0)
    cv2.putText(result, 'Curvature = {:.0f} (m)'.format(min(left_line.radius_of_curvature, right_line.radius_of_curvature)), org=(10, 100), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,color=(255, 255, 255), thickness=2)
    cv2.putText(result, 'Vehicle is {:.2f} (m) away from center'.format(pos), org=(10, 140), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,color=(255, 255, 255), thickness=2)    
    return result

In [17]:
# Measure curvature
def measure_curvature_real(ploty):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_line.best_fit[0]*y_eval*ym_per_pix + left_line.best_fit[1])**2)**1.5) / np.absolute(2*left_line.best_fit[0])
    right_curverad = ((1 + (2*right_line.best_fit[0]*y_eval*ym_per_pix + right_line.best_fit[1])**2)**1.5) / np.absolute(2*right_line.best_fit[0])

    # Calculate distance
    xl = np.dot(left_line.best_fit, [700**2, 700, 1])
    xr = np.dot(right_line.best_fit, [700**2, 700, 1])
    pos = (1280//2 - (xl+xr)//2)*xm_per_pix
    
    return left_curverad, right_curverad, pos

## Test lane drawing

In [18]:
# Save the result to the test_images_output directory.
images = glob.glob("test_images/" + '*.jpg')
left_line = Line()
right_line = Line()

for fname in images:
    # Get initial 
    img = cv2.imread(fname)
    final = draw_lane_area(img)
    image_compare('Detected_lanes_'+ fname[12:], img, final, 'output_images/draw_lane_area/', None)

## Test project main video

In [19]:
output = 'output_videos/project_video_result.mp4'
clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(0,30)

# Create line instances
left_line = Line()
right_line = Line()

white_clip = clip1.fl_image(draw_lane_area)
%time white_clip.write_videofile(output, audio=False)

t:   0%|          | 0/750 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/project_video_result.mp4.
Moviepy - Writing video output_videos/project_video_result.mp4



Moviepy - Done !
Moviepy - video ready output_videos/project_video_result.mp4
CPU times: user 10min 5s, sys: 53 s, total: 10min 58s
Wall time: 2min 55s


In [20]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

In [21]:
output_challange = 'output_videos/challenge_video_result.mp4'
clip2 = VideoFileClip("test_videos/challenge_video.mp4").subclip(0,30)
# Create line instances
left_line = Line()
right_line = Line()
white_clip_2 = clip2.fl_image(draw_lane_area)
%time white_clip_2.write_videofile(output_challange, audio=False)

t:   0%|          | 0/900 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/challenge_video_result.mp4.
Moviepy - Writing video output_videos/challenge_video_result.mp4



Moviepy - Done !
Moviepy - video ready output_videos/challenge_video_result.mp4
CPU times: user 12min 7s, sys: 1min 2s, total: 13min 9s
Wall time: 3min 28s


In [22]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_challange))

In [23]:
output_harder = 'output_videos/harder_challenge_video_result.mp4'
clip3 = VideoFileClip("test_videos/harder_challenge_video.mp4").subclip(0,30)
left_line = Line()
right_line = Line()
white_clip_3 = clip3.fl_image(draw_lane_area)
%time white_clip_3.write_videofile(output_harder, audio=False)

t:   0%|          | 0/750 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/harder_challenge_video_result.mp4.
Moviepy - Writing video output_videos/harder_challenge_video_result.mp4



Moviepy - Done !
Moviepy - video ready output_videos/harder_challenge_video_result.mp4
CPU times: user 10min 19s, sys: 54.8 s, total: 11min 14s
Wall time: 3min 7s


In [24]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_harder))